In [1]:
import numpy as np
import pandas as pd
import h5py
import glob
from pprint import pprint
from skimage.measure import regionprops
import time
import matplotlib.pyplot as plt

In [2]:
def file_properties(path_file):
    with h5py.File(path_file, 'r') as f:
        if 'label' not in f.keys():
            message_err = "'label' dataset must be included in the input HDF5 file! " + \
                          f"Only found the following keys:\n{f.keys()}"
            raise KeyError(message_err)

        time_start = time.time()
        dset = f['label'][:]
        time_end = time.time()
        print(f"Load time: {time_end - time_start}")

        regs = regionprops(dset)

        # properties = [dset.shape, dset.dtype, ]

    return regs

In [3]:
def get_files_from_folder(path_folder, ext='h5'):
    pathlist_files = glob.glob(path_folder + '/*.' + ext)
    pprint(pathlist_files)
    return(pathlist_files)

In [47]:
path_folder = "/g/kreshuk/yu/Datasets/TMody2021Ovules/train"
pathlist_files = get_files_from_folder(path_folder, ext='h5')
regs = file_properties(pathlist_files[0])

['/g/kreshuk/yu/Datasets/TMody2021Ovules/train/1613.h5',
 '/g/kreshuk/yu/Datasets/TMody2021Ovules/train/1614.h5',
 '/g/kreshuk/yu/Datasets/TMody2021Ovules/train/1616.h5',
 '/g/kreshuk/yu/Datasets/TMody2021Ovules/train/1617.h5']
Load time: 3.4438726902008057


In [57]:
extents = np.array([np.array(r.bbox[3:]) - np.array(r.bbox[:3]) for r in regs])
areas = np.array([r.area for r in regs])
median_extents = np.median(extents, axis=0)
anisotropy = tuple(np.max(median_extents) / median_extents)
grid = tuple(1 if a > 1.5 else 2 for a in anisotropy)

In [59]:
anisotropy, grid, np.max(median_extents)

((4.2105263157894735, 1.0256410256410255, 1.0), (1, 2, 2), 80.0)

In [45]:
def area_distribution(areas):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))
    
    ax.hist(areas)
    ax.set_title('Number of Pixels')
    
    fig.tight_layout()

In [34]:
def size_distribution(extents):
    fig, axs = plt.subplots(3, 1, figsize=(10, 15))
    dims = ['z', 'y', 'x']

    fig.suptitle("Distribution of Object Sizes")
    for idx in range(3):
        axs[idx].hist(extents[:, idx])
        axs[idx].set_title(dims[idx])

    fig.tight_layout()

In [ ]:
def check_dataset_names(path_file):
    with h5py.File(path_file, 'r') as f:
        print(f.keys())
    return


def get_files_from_folder(path_folder, ext='h5'):
    pathlist_files = glob.glob(path_folder + '/*.' + ext)
    pprint(pathlist_files)
    return(pathlist_files)


def file_properties(path_file):
    with h5py.File(path_file, 'r') as f:
        if 'label' not in f.keys():
            message_err = "'label' dataset must be included in the input HDF5 file! " + \
                          f"Only found the following keys:\n{f.keys()}"
            raise KeyError(message_err)

        time_start = time.time()
        dset = f['label'][:]
        time_end = time.time()
        print(f"Load time: {time_end - time_start}")

        regs = regionprops(dset)

        # properties = [dset.shape, dset.dtype, ]

    return regs


def understand_dataset(path_folder, ext='h5'):
    pathlist_files = get_files_from_folder(path_folder, ext='h5')
    for path_file in pathlist_files:
        file_properties(path_file)
    return


if __name__ == '__main__':
    path_folder = "/g/kreshuk/yu/Datasets/TMody2021Ovules/train"
    pathlist_files = get_files_from_folder(path_folder, ext='h5')
    # understand_dataset(path_folder)
    regs = file_properties(pathlist_files[0])
    extents = np.array([np.array(r.bbox[3:]) - np.array(r.bbox[:3]) for r in regs])
    areas = np.array([r.area for r in regs])
    plt.hist(areas, bin=50)
    print(regs[0].bbox, regs[0].area)
